In [1]:
import contracts
contracts.disable_all()

In [2]:
import duckietown_uplan as uplan

INFO:dt-uplan:duckietown_uplan 1.0.0
INFO:dt-world:duckietown-world 1.0.19
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class GenericData
DEBUG:dt-serialization:Registering class Sequence
DEBUG:dt-serialization:Registering class SampledSequence
DEBUG:dt-serialization:Registering class Constant
DEBUG:dt-serialization:Registering class RectangularArea
DEBUG:dt-serialization:Registering class TransformSequence
DEBUG:dt-serialization:Registering class VariableTransformSequence
DEBUG:dt-serialization:Registering class SE2Transform
DEBUG:dt-serialization:Registering class Scale2D
DEBUG:dt-serialization:Registering class Matrix2D
DEBUG:dt-serialization:Registering class SpatialRelation
DEBUG:dt-serialization:Registering class GroundTruth
DEBUG:dt-serialization:Registering class PlacedObject
DEBUG:dt-serialization:Registering class EvaluatedMetric
DEBUG:dt-serialization:Registering class GenericObject
DEBUG:dt-serialization:Registering class Duckie
D

In [3]:
import duckietown_world as dw
current_map = dw.load_map('4way')
simulation_exp = uplan.ConstantProbabiltiySim(current_map, 3)

INFO:dt-world:loading map 4way


In [4]:
# simulation_exp.render_current_state()
max_rad = simulation_exp.duckie_town.duckie_citizens[0].get_max_radius()
current_graph = simulation_exp.duckie_town.get_current_graph()

In [14]:
from duckietown_uplan.environment.footprint_table import FootprintTable
foot_print_table = FootprintTable(current_graph, max_rad)
data = foot_print_table.get_data()

AttributeError: 'FootprintTable' object has no attribute 'get_data'